## 網路爬蟲

In [9]:
import requests
res = requests.get('https://rent.591.com.tw/?kind=1&region=1&section=5')
res
#res.text

<Response [200]>

## 使用BeautifulSoup取得Token

In [10]:
import requests
from bs4 import BeautifulSoup
rs = requests.session()
res = rs.get('https://rent.591.com.tw/?kind=1&region=1&section=5')
soup = BeautifulSoup(res.text, 'lxml')
token = soup.select_one('meta[name="csrf-token"]')['content']

In [11]:
token

'LCJQBZv8qek9xGMNbhdlz5FX2Ego4Let1tbOXNKn'

## 串接 API 取得資料

In [12]:
headers = {
'X-CSRF-TOKEN': token,
'X-Requested-With': 'XMLHttpRequest'    
    
}

apiurl = 'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=1&searchtype=1&region=1&section=5&firstRow=30&totalRows=789'
res = rs.get(apiurl, headers = headers)
jd = res.json()

## 資料清理

In [13]:
import pandas
df = pandas.DataFrame(jd['data']['data'])
df.head()

,addInfo,addition2,addition3,addition4,addr_number_name,address,address_img,address_img_title,alley_name,allfloor,...,storeprice,street_name,streetid,type,unit,updatetime,user_id,vipBorder,vipimg,vipstyle
0,"<img src=""./images/index/userCenter/list_vip_v...",0,0,0,,和平東路三段麟光站旁●精..,麟光站旁●精美屋況●採光明亮●屋齡新,麟光站旁●精美屋況●採光明亮●屋齡新,,14,...,0,和平東路三段,25634,1,元/月,1559299637,1611062,vipStyle,"<img src=""./images/index/userCenter/list_vip_v...",isvip
1,"<img src=""./images/index/userCenter/list_vip_v...",0,0,0,,忠孝東路三段251巷忠孝復..,忠孝復興只要一分鐘,忠孝復興只要一分鐘,251巷,4,...,0,忠孝東路三段,25636,1,元/月,1559926715,2592722,vipStyle,"<img src=""./images/index/userCenter/list_vip_v...",isvip
2,"<img src=""./images/index/userCenter/list_vip_v...",0,0,0,,敦化南路二段24H管理正面..,24H管理正面敦化南路林蔭,24H管理正面敦化南路林蔭,,26,...,0,敦化南路二段,25663,1,元/月,1558658859,518411,vipStyle,"<img src=""./images/index/userCenter/list_vip_v...",isvip
3,"<img src=""./images/index/userCenter/list_vip_v...",0,0,0,202號,大安路一段仁愛A+大安捷運..,仁愛A+大安捷運站精美套房(含管理費),仁愛A+大安捷運站精美套房(含管理費),,12,...,0,大安路一段,25614,1,元/月,1559705961,60453,vipStyle,"<img src=""./images/index/userCenter/list_vip_v...",isvip
4,"<img src=""./images/index/userCenter/list_vip_v...",0,0,0,24號,四維路160巷遠企大樓對面..,遠企大樓對面，建安國小旁，3房2廳2衛,遠企大樓對面，建安國小旁，3房2廳2衛,160巷,4,...,0,四維路,25619,1,元/月,1559824683,1083535,vipStyle,"<img src=""./images/index/userCenter/list_vip_v...",isvip


In [14]:
df = df[['address', 'floor', 'allfloor', 'layout', 'area', 'price', 'browsenum_all']]
df.head()

,address,floor,allfloor,layout,area,price,browsenum_all
0,和平東路三段麟光站旁●精..,2,14,4房2廳2衛,46.0,"55,000",218
1,忠孝東路三段251巷忠孝復..,2,4,3房2廳2衛,27.0,"36,000",1770
2,敦化南路二段24H管理正面..,13,26,2房2廳2衛,41.1,"49,800",647
3,大安路一段仁愛A+大安捷運..,5,12,1房1廳1衛,10.6,"26,000",679
4,四維路160巷遠企大樓對面..,4,4,3房2廳2衛,30.0,"34,000",2647


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
address          30 non-null object
floor            30 non-null int64
allfloor         30 non-null int64
layout           30 non-null object
area             30 non-null float64
price            30 non-null object
browsenum_all    30 non-null int64
dtypes: float64(1), int64(3), object(3)
memory usage: 1.7+ KB


In [18]:
df['price'] = df['price'].map(lambda e: int(e.replace(',', '')))

In [20]:
df[['layout_bedroom','layout_livingroom', 'layout_bathroom']] = df['layout'].str.extract('(\d+)房(\d+)廳(\d+)衛')

In [21]:
df['layout_bedroom'] = df['layout_bedroom'].astype(int)
df['layout_livingroom'] = df['layout_livingroom'].astype(int)
df['layout_bathroom'] = df['layout_bathroom'].astype(int)

In [22]:
#del df['layout']
df.drop('layout', axis = 1, inplace=True)

In [23]:
df.head()

,address,floor,allfloor,area,price,browsenum_all,layout_bedroom,layout_livingroom,layout_bathroom
0,和平東路三段麟光站旁●精..,2,14,46.0,55000,218,4,2,2
1,忠孝東路三段251巷忠孝復..,2,4,27.0,36000,1770,3,2,2
2,敦化南路二段24H管理正面..,13,26,41.1,49800,647,2,2,2
3,大安路一段仁愛A+大安捷運..,5,12,10.6,26000,679,1,1,1
4,四維路160巷遠企大樓對面..,4,4,30.0,34000,2647,3,2,2


## 資料儲存

In [24]:
df.to_csv('rent591.csv')

In [25]:
df.to_excel('rent591.xlsx')

In [26]:
import sqlite3
with sqlite3.connect('house.sqlite') as db:
    df.to_sql('renthouse', con = db)

In [27]:
with sqlite3.connect('house.sqlite') as db:
    df2= pandas.read_sql_query('select * from renthouse order by price desc ', con = db)
df2.head()

,index,address,floor,allfloor,area,price,browsenum_all,layout_bedroom,layout_livingroom,layout_bathroom
0,6,安和路二段純住名邸●低公..,11,25,62.4,95000,21,3,2,2
1,7,和平東路二段175巷..,2,7,48.0,80000,414,4,2,2
2,20,青田街古亭師大圈.生活機..,5,6,52.0,75000,185,2,2,2
3,16,建國南路一段大安森林公園..,8,13,37.8,60000,184,3,2,2
4,17,信義路二段*近中正纪念堂..,3,14,49.3,60000,1150,3,2,2


## 完整591網路爬蟲

In [28]:
import requests
from bs4 import BeautifulSoup
rs  = requests.session()
res = rs.get('https://rent.591.com.tw/?kind=1&region=1&section=5')
soup = BeautifulSoup(res.text)
token = soup.select_one('meta[name="csrf-token"]')['content']

In [29]:
import pandas
headers = {
'X-CSRF-TOKEN': token,
'X-Requested-With': 'XMLHttpRequest'    
}
apiurl = 'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=1&searchtype=1&region=1&section=5&firstRow={}&totalRows=793'

pagenum = int(793 / 30) + 1
dataary = []
for p in range(pagenum):
    res2 = rs.get(apiurl.format(p*30), headers = headers)
    df = pandas.DataFrame(res2.json()['data']['data'])
    dataary.append(df)

In [30]:
import pandas
df = pandas.concat(dataary)
df.head()

,addInfo,addition2,addition3,addition4,addr_number_name,address,address_img,address_img_title,alley_name,allfloor,...,storeprice,street_name,streetid,type,unit,updatetime,user_id,vipBorder,vipimg,vipstyle
0,,0,0,1,23號,忠孝東路四段181巷東區靜..,東區靜巷公寓2樓,東區靜巷公寓2樓,181巷,5,...,0,忠孝東路四段,25637,1,元/月,1559744184,1380158,vipStyle,,isvip
1,,0,0,1,1號,復興南路160巷精華地段雙..,精華地段雙捷運．新落成SRC鋼骨制震豪宅,精華地段雙捷運．新落成SRC鋼骨制震豪宅,160巷,15,...,0,復興南路,64718,1,元/月,1560249787,890339,vipStyle,,isvip
2,,0,0,1,220號,"臥龍街前眺101後擁山景,..","前眺101後擁山景,捷運5分鐘,總裁住家","前眺101後擁山景,捷運5分鐘,總裁住家",,12,...,0,臥龍街,25640,1,元/月,1559719922,229230,vipStyle,,isvip
3,,0,0,1,86號,光復南路高樓層大巨蛋正對..,高樓層大巨蛋正對面近捷運國父紀念館站,高樓層大巨蛋正對面近捷運國父紀念館站,,13,...,0,光復南路,25623,1,元/月,1558932105,452502,vipStyle,,isvip
4,,0,0,1,,信義路三段台北大安森林公..,台北大安森林公園(可短租/設籍/營登),台北大安森林公園(可短租/設籍/營登),,14,...,0,信義路三段,25647,1,元/月,1556941550,104199,vipStyle,,isvip


In [31]:
df = df[['address', 'floor', 'allfloor', 'layout', 'area', 'price', 'browsenum_all']]

In [32]:
df['price'] = df['price'].map(lambda e: int(e.replace(',', '')))

In [33]:
df[['layout_bedroom','layout_livingroom','layout_bathroom']] = df['layout'].str.extract('(\d+)房(\d+)廳(\d+)衛')

In [35]:
df.dropna(inplace=True)

In [36]:
df['layout_bedroom'] = df['layout_bedroom'].astype(int)
df['layout_livingroom'] = df['layout_livingroom'].astype(int)
df['layout_bathroom'] = df['layout_bathroom'].astype(int)

In [37]:
del df['layout']

In [38]:
import sqlite3
with sqlite3.connect('house.sqlite') as db:
    df.to_sql('renthouse', con = db, if_exists='replace')

In [39]:
df.to_excel('rent591.xlsx')